In [1]:
# !pip install jiwer -qq

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from sk import predict
import onnxruntime
import onnx
from pathlib import Path
import jiwer
import pandas as pd

In [4]:
model = onnx.load("/content/neem/onnx/conformer_small.onnx")
model = onnxruntime.InferenceSession(model.SerializeToString())

# Predict single or multiple file

In [5]:
fn = "/content/test/youtube/0228444ff34081eda587c0ca53712486.wav"
preds,files,entropy,timesteps = predict(fn,model)

  0%|          | 0/1 [00:00<?, ?it/s]

Total input path: 1
Total audio found(.wav): 1
start prediction


100%|██████████| 1/1 [00:01<00:00,  1.17s/it]


In [6]:
Path(fn).with_suffix(".txt").read_text()

'pada peringkat ini pembukaan sekolah hanya melibatkan'

In [7]:
preds[0],entropy[0].item()

('pada peringkat ini pembukaan sekolah hanya melibatkan', 0.08121918141841888)

# Predict from single or multiple folder

In [8]:
fn = "/content/test/"
preds,files,entropy,timesteps = predict(fn,model)

  0%|          | 0/243 [00:00<?, ?it/s]

Total input path: 1
Total audio found(.wav): 243
start prediction


100%|██████████| 243/243 [00:24<00:00,  9.86it/s]


In [14]:
data = []
for i,j in zip(preds,files):
    label = j.with_suffix('.txt').read_text()
    data.append([j,label,i,jiwer.compute_measures(label,i)['wer'],jiwer.compute_measures([l for l in label],[ii for ii in i])['wer']])

In [15]:
df = pd.DataFrame(data)
df.columns = ["path","label","pred","wer","cer"]
df.head()

,path,label,pred,wer,cer
0,/content/test/youtube/63e3274518dd294e85f7c374...,belum menduduki sebarang peperiksaan okey ada ...,belum menduduki sebarang peperiksaan okey ada ...,0.25,0.145833
1,/content/test/youtube/0ca710eca52e9a83635d71ab...,duduklah aha ha,adalah,1.00,0.692308
2,/content/test/youtube/a07a8d2830fed6593449ceda...,sedia ada untuk penggal persekolahan yang kita...,sedia ada untuk penggal persekolahan yang kita...,0.00,0.000000
3,/content/test/youtube/7c8a921f6d3b20b94ec3a7b8...,assalamualaikum warahmatullah,assalamualaikum watmatullah,0.50,0.107143
4,/content/test/youtube/0872da274c3e1e24df22b58d...,apahal biasalah pak yah kes pak cik bahar lah tu,biasalah pakya kes pak cibahallah tu,0.60,0.230769


In [17]:
df["wer"].mean(),df["cer"].mean()

(0.24842461021815934, 0.0934487866621077)